### Previously

In the previous section we covered the bigram langauge model. We only loked at single previous character, and we predicted the distrib ution for the character that would follwo that. This is all good if we have on character of previous context. The problem with thismodel is that the predictions are not very good. becasue we take one character of previous context. If we take more context into account the size of the probability table grows expotnentiall and becomes out of control. If we take 1 character we have 27 rows in the prob table/heatmap. If we take 2 cahracter in the context we have 27*27=729 possiblities of what could have come in the context. if we take 3 characters in the context we have 27*27*27 combination and possiblities. This causes the possiblities to explode. 

### In this Notebook

We will impelent the multilayer perceptron (MLP) follwoing the Bengio et al. 2003 paper. But we will do it for cahracters instead of words. The paper can be found here https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf.
This is not the first paper to come up with this model/idea. But its the one that is mostly cited and is a well written paper. Keep in mind that we are doing a character level language model. In the paper they have a vocabulary of 17000 words and they insted build a word level language model. We will stick to chars but will stick to the same model. Their proposal is to take every one of the 17000 words and associated with each word a 30 dim feature vector. Everyword is embedded into a 30 dim space. We have 17000 points in a 30 dimesnsional space. A lot of points for a very small space. Initially these words are initialized randomly. Then we are going to tune the embeddings of these words through backpropagation. during the course of training of this NN these points or vectors are going to move around in the space. Words that are similar will end up in a similar part of space and words that mean very different from each other will end up in less similar spaces in the vector space. They are using a multilayer NN to predict the next word given the previous ones. And in the paper they are maximizing the log-likelihood of the trianing data. 

why does it work? There might be sentences that did not occur in the trianing data at all. In other words we have out of distribution cases. The model looks at similar embeddings and can deal with out of distribution cases. Model is deployed somewhere and is trying to make a sentence say "A dog was running in a room". Because the model was not trained on this, this case is an out of distribution one. maybe the model has seen a similar case like "the dog ...". maybe the model has learned that "a" and "the" and tooke the embedding for each and put them closer to each other in the embeddeing space. So you can transfer knowledge through the similar embeddings and you can generalize in that way. SImilarly the NN can know that cats and dogs are animals and they co occur in lots of similar context. So it can transfer knowledge regarding that through the embedding space and  you can generalize to novel scenarios




Looking at the Figure 1 of the paper: we are taking three previous words and we are trying to predict the 4th in tehsequence. The 3 previuos words we have vocab of 17000 possible words, so veryone of these basically are the index of the incoming words. because there are 17000 words the index input (index for w_t-n+1) is an integer between 0 and 16999. Thre is also a lookup table that they call "C" in the paper. and the lookup table is a matrix of 17000X30. and we are treating it as a lookup table for the index. every index is plucking out a row of this matrix. So each word corresponds to an index and the index can be looked up from the lookup table to get the vector embedding for that specific word. In the figure we have 30 neurons for 3 words making up 90 neurons in total. The matrix 'C' is shared acros all the words. We are plucking out from the same matrix C. Next step we have the hidden layer of the NN indicated by tanh. The size of the hidden layer is a hyperparameter. it can be as large or as small as we want. We go over multiple choices of the size of this NN and se which one works best. Say there woere 100 nuerons in the hidden layer and all of them are fully connected to the 90 numbers that make up the 3 words. So we have a fully connected layer, then there is a tanh non-linearity. Then there is the output layer and beacseu there are 17000 possible words that could come next this layer has 17000 neurons and all of them are fully connected to all the neurons in the hidden layer. So there is a lot of parameters because there is a lot of words. So most of the computation happens in the final layer. So there are 17000 logits and on top of that we have the softmax layer which we have seen before. everyone of lgits is exponentiated and everyone of them is normalized to sum to 1 so that we have a nice prob distribution for the next word in the sequence. During trianing we actually have the lalbel, or the identity of the next words inthe sequence. That word or its index is used to pluck out the probability of that word and then we are maximizing the probability of that word wrt the parameters of the Neural Net. The parameters are the weights and biases of the output layer, the weights and biases of the hidden layer, and the embedding lookup table "C". All of that is optimized using backpropagation. The dashed arrows in the Fig 1 represent a variation of a nueral net that we will ignore for now.

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [1]:
# read in all the words
words = open ("./names.txt", 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [2]:
len (words)

32033